In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
from openslide import OpenSlide
import openslide 
from openslide import open_slide
import math
import uuid
import PIL.Image
import PIL.ImageDraw
from labelme.logger import logger
import json
import glob

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
from openslide import OpenSlide
import openslide 
from openslide import open_slide
import math
import uuid
import PIL.Image
import PIL.ImageDraw
from labelme.logger import logger
import json
import glob

#　JSONファイルからマスク画像へと変換する関数の定義
def shape_to_mask(
 img_shape, points, shape_type=None, line_width=10, point_size=5
):
    mask = np.zeros(img_shape[:2], dtype=np.uint8)
    mask = PIL.Image.fromarray(mask)
    draw = PIL.ImageDraw.Draw(mask)
    xy = [tuple(point) for point in points]
    if shape_type == "circle":
        assert len(xy) == 2, "Shape of shape_type=circle must have 2 points"
        (cx, cy), (px, py) = xy
        d = math.sqrt((cx - px) ** 2 + (cy - py) ** 2)
        draw.ellipse([cx - d, cy - d, cx + d, cy + d], outline=1, fill=1)
    elif shape_type == "rectangle":
        assert len(xy) == 2, "Shape of shape_type=rectangle must have 2 points"
        draw.rectangle(xy, outline=1, fill=1)
    elif shape_type == "line":
        assert len(xy) == 2, "Shape of shape_type=line must have 2 points"
        draw.line(xy=xy, fill=1, width=line_width)
    elif shape_type == "linestrip":
        draw.line(xy=xy, fill=1, width=line_width)
    elif shape_type == "point":
        assert len(xy) == 1, "Shape of shape_type=point must have 1 points"
        cx, cy = xy[0]
        r = point_size
        draw.ellipse([cx - r, cy - r, cx + r, cy + r], outline=1, fill=1)
    else:
        assert len(xy) > 2, "Polygon must have points more than 2"
        draw.polygon(xy=xy, outline=1, fill=1)
    mask = np.array(mask, dtype=bool)
    return mask



#pathはpost_annotetion_imageのディレクトリ
#下記でfiles:ディレクトリ内にあるJSONファイルの絶対パスリストを　filesに格納.files_nanozoomerに元画像ディレクトリパス（★）を格納
#ディレクトリの中のJSONファイルだけを取得
path = r"F:\AI byouri\pre_annotation_image\pre_annotation_image__ly_neg__"
files=[]
for name in glob.glob(r"F:\AI byouri\pre_annotation_image\pre_annotation_image__ly_neg__/*.json"):
    files.append(name)
# print(files)
#files1の中にアノテーションして得たJSONファイルの相対パスを得る。後でJSONファイルの検体ナンバーを知るのに必要
files1=[]
files0 = os.listdir(r"F:\AI byouri\pre_annotation_image\pre_annotation_image__ly_neg__")
for a in files0:
    if "json" in a:
        files1.append(a)
# print(files1)
    
# ナノズーマー取り込み画像のフォルダ名前パスを取得
files_nanozoomer=glob.glob(r"F:\AI byouri\ndp_image\virtual slide_Ly meta- wall")
# print(files_nanozoomer) # 絶対パス

for m,n in enumerate(files):
    print(m)    
    
#以下でJOSONファイルからアノテーション画像を呼び出す。
    with open(n) as f:
        dj = json.load(f)
# dj['shapes'][0]は今回一つのラベルのため。
    mask = shape_to_mask((dj['imageHeight'],dj['imageWidth']), dj['shapes'][0]['points'], shape_type=None,line_width=1, point_size=1)
    mask_img = mask.astype(np.int)#booleanを0,1に変換
    width_mask,height_mask=mask_img.shape
#     print(mask_img.shape)
    #　mask_imgは元画像を1/50にして　アノテーションしたもの
    #もともとのバーチャルスライド画像を呼び出し
    re_files=files1[m]
    mask_number=re_files[:4]
    print(mask_number)
    #このmask_numberが症例番号にあたり、それと一致する　nanozoomer取り込み画像を下でWSIに格納する
    wsi = [s for s in files_nanozoomer if mask_number in s]
    
    WSI=r"{}".format(wsi[0])
    print(WSI)
    
    osimg = OpenSlide(WSI)
    width,height=osimg.dimensions
    width_mask,height_mask=mask_img.shape
    print(width,height)
    print(width_mask,height_mask)
    #最後に症例ごとにtumor　notumorをわけるディレクトリ作っておく
    base_dir_tumor=r"D:\noda\noda\learning_data_set\add_ly+_tumor"
    mm="{}".format(n)
    new_dir_tumor = base_dir_tumor +"\\"+ mask_number
    os.mkdir(new_dir_tumor)
    new_dir2_tumor="r"+'"{}"'.format(new_dir_tumor)#windowsだからこの一文が必要
    print(new_dir_tumor) 
    
    
    base_dir_no_tumor=r"D:\noda\noda\learning_data_set\add_ly+_no_tumor"
    mm="{}".format(n)
    new_dir_no_tumor = base_dir_no_tumor +"\\"+ mask_number
    os.mkdir(new_dir_no_tumor)
    new_dir2_no_tumor="r"+'"{}"'.format(new_dir_no_tumor)#windowsだからこの一文が必要
    print(new_dir_no_tumor) 
    
    #size1,2を用意。size１はアノテーション画像を分割するため、size2は元画像を分割するため。
    size= 2000
    size1=40
    #dementionsでndpi画像のsizeとwidthを格納
    #以下は2000で分割
    #割り切れる数に調整
    height = height // size * size
    width = width // size * size
    height_mask = height_mask // size1 * size1
    width_mask = width_mask // size1 * size1
    #何個に分割するかを変数に入れる
    height_split = height//size
    width_split = width//size
    height_mask_split = height_mask // size1 
    width_mask_split = width_mask // size1
    #　新しいイメージの高さ、幅を変数に入れる(基本的にはsizeと一緒になる)
    new_img_height = int(height / height_split)
    new_img_width = int(width / width_split)
    new_img_height_mask = int(height_mask / height_mask_split)
    new_img_width_mask = int(width_mask / width_mask_split)
    for h in range(height_split):
        height_mask_start = h * new_img_height_mask
        height_mask_end = height_mask_start + new_img_height_mask
        l=2000*h
        for w in range(width_split):
            width_mask_start = w * new_img_width_mask
            width_mask_end = width_mask_start + new_img_width_mask
            m=2000*w
            img = osimg.read_region((m,l), 0, (2000,2000))
            print(img)
            img = np.array(img, dtype=np.uint8)
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img_gray_mean=np.mean(img_gray)
            #まず背景が白い画像を省く。　閾値を調整すれば、どれくらいの白い画像を省けるかを調整できる。ひとまず２２０にした。
#             img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            if img_gray_mean > 200:
                continue
            clp = mask_img[height_mask_start:height_mask_end,width_mask_start:width_mask_end]
            clp_mean=np.mean(clp)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)       
            #マスクされてない非腫瘍を　no_tumorフォルダへ格納
            
            if clp_mean==0:
                file_name = "no_tumor" + str(h) + "_" + str(w) + ".png"
                folded = new_dir_no_tumor
                cv2.imwrite(folded+"/" + file_name, img)
            #マスクされた腫瘍を　tumorフォルダへ格納
            else:
                file_name = "tumor" + str(h) + "_" + str(w) + ".png"
                folded =new_dir_tumor
                cv2.imwrite(folded+"/" + file_name, img)








0
0010
D:\noda\noda\NDP_image\add_posi_N23\tsuika_N23\0010-18 - 2020-12-06 10.55.28.ndpi
143360 95488
1909 2866
D:\noda\noda\learning_data_set\add_ly+_tumor\0010
D:\noda\noda\learning_data_set\add_ly+_no_tumor\0010
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194

<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154178E7880>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193CC520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image

<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193CC4F0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193CC4F0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193CC4F0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154194EA6A0>
<PIL.Image

<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D84C0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154178E76A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D84C0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154178E7310>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D84C0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154178E7880>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D84C0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154178E72B0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D84C0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154178E75E0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D84C0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154178E76A0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D84C0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154178E75E0>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D84C0>
<PIL.Image

<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image

...

<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image

<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image

<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image

<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image

<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image.Image image mode=RGBA size=2000x2000 at 0x154193D8520>
<PIL.Image

In [2]:
pip install opencv

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement opencv (from versions: none)
ERROR: No matching distribution found for opencv
